In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Define paths to your train and test datasets
train_data_dir = '/content/drive/MyDrive/cancerdata/TrainValidation'
test_data_dir = '/content/drive/MyDrive/cancerdata/Test'

# Parameters
img_width, img_height = 299, 299
batch_size = 64
num_epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for training and testing data with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


test_datagen = ImageDataGenerator(rescale=1./255)

# Reinitialize generators with data augmentation
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')




# Load InceptionV3 model pretrained on ImageNet dataset
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(1024, activation='relu')(x)

# Add a logistic layer for the number of classes
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze all InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history=model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
    )
# Sauvegarder le modèle après l'entraînement
model.save("/content/drive/MyDrive/Models/incV3.h5")


In [ ]:
accuracy = history.history['val_accuracy']
print("Accuracy:", accuracy)  # Print the final accuracy


In [ ]:
validation_accuracy_last_epoch = history.history['val_accuracy'][-1]
print("Validation Accuracy of the last epoch:", validation_accuracy_last_epoch)

In [ ]:
import matplotlib.pyplot as plt

# Plot loss and accuracy
plt.figure(figsize=(10, 6))

# Plot Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')

plt.xlabel('Epoch')
plt.ylabel('Value')
plt.title('Training and Validation Loss/Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator)
print("Test Accuracy:", accuracy)
print("test loss :",loss)